<a href="https://colab.research.google.com/github/hareidx/FASTAI_HAREID/blob/main/FASTAI_Week_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import socket, warnings
try:
  socket.setdefaulttimeout(1)
  socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
  print(f'internet connection is fine')
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in the top right and set 'Internet' Switch to on")

internet connection is fine


In [2]:
!pip install -Uqq fastai

     |████████████████████████████████| 197 kB 4.3 MB/s 
     |████████████████████████████████| 60 kB 5.5 MB/s 
